In [ ]:
# Libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import Javascript, display, HTML
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None

In [ ]:
# Read the input file
dfmain = pd.read_csv('gtrends_opportunity_test_2022-07-27(1).csv', decimal=',')
df = dfmain[['Item Code', 'Price Point', 'Lowest Selling Price', 'Highest Selling Price', 
             'Category', 'Stock Qty.', 'gtrends_opportunity', 'ai_keyword', 'Image']]

In [ ]:
# Add the images
def image_match(keyword):
    for file in os.listdir('images'):
        try:
            if keyword in file:
                return f'images\{file}'
        except:
            pass

df['Image'] = df['ai_keyword'].apply(lambda x: image_match(x))

In [ ]:
# Widgets
category_widget = widgets.Dropdown(
    options=df["Category"].unique(),
    value=df["Category"].unique()[1],
    description="Category",
    disbaled = False
    )

price_widget = widgets.IntRangeSlider(
    value=[0, 10],
    min=0,
    max=df['Highest Selling Price'].max(),
    step=1,
    description='Price($):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    color='LimeGreen'
)
    
stock_widget = widgets.IntSlider(
    value=0,
    min=0,
    max=df["Stock Qty."].max(),
    step=1,
    description='Stock Qty.',
    disabled=False,
    continuous_update=False,
    handle_color= 'lightblue'
)

# Choose the no. of rows to display
n_widget = widgets.IntSlider(
    value=1,
    min=1,
    max=20,
    step=1,
    description='No. records',
    disabled=False,
    continuous_update=False
)

In [ ]:
# Convert links to thumbnail images
def path_to_image(path):
    return '<img src="'+ path + '" width="100" >'

In [ ]:
# Filtering
def show_df(n=n_widget, category=category_widget, stock=stock_widget, price=price_widget):
    df_filtered = df[(df["Category"]==category) &
                   (df["Stock Qty."]>stock) &
                   (df['Lowest Selling Price']>=list(price)[0]) &
                   (df['Highest Selling Price']<=list(price)[1])].head(n)
    df_final = df_filtered.drop(['Lowest Selling Price', 'Highest Selling Price'], axis=1)
    df_final = df_final.sort_values(by=['gtrends_opportunity'], ascending=False, ignore_index=True)    
    display(HTML(df_final.to_html(escape=False, formatters=dict(Image=path_to_image))))

In [ ]:
# Display
web_app = widgets.interactive(show_df)
display(web_app)

In [ ]:
# End